In [1]:
import pandas as pd
import numpy as np
# import time
# import itertools

In [ ]:
# !!!

# from ... import to_split_and_scale

<a id="1"></a>

# 1. Downloading data

In [2]:
price_df4 = pd.read_csv("poland_apartments_completed.csv")
price_df4.head()

,Unnamed: 0,id,city,floor,latitude,longitude,price,rooms,sq,year,latitude_center,longitude_center,decade,price_per_m,sq_round,radius,radius_dec,radius_ceil,district
0,0,23918.0,Kraków,2,50.049224,19.970379,749000,3,74,2021,50.061389,19.937222,2020,10121,70,2.732975,2.7,3,Dzielnica XIII Podgórze
1,1,17828.0,Warszawa,3,52.249775,21.106886,240548,1,24,2021,52.230000,21.011111,2020,10022,20,6.902500,6.9,7,Praga Południe
2,2,22784.0,Kraków,2,50.066964,19.920025,427000,2,37,1970,50.061389,19.937222,1970,11540,30,1.378660,1.4,2,Dzielnica V Krowodrza
3,3,4315.0,Poznañ,2,52.404212,16.882542,1290000,5,166,1935,52.408333,16.933611,1930,7771,160,3.505600,3.5,4,Grunwald
4,4,11770.0,Warszawa,1,52.212225,20.972630,996000,5,105,2020,52.230000,21.011111,2020,9485,100,3.290526,3.3,4,Ochota


In [3]:
cat_features = ['city', 'district'] # 'floor', 'rooms'?
num_features = ['floor', 'rooms', 'sq', 'year', 'radius']
target = ['price']


In [ ]:
# WHAT NEEDS FOR LAUNCHING:

# price_df4 # +
# cat_features # +
# labels_dict # ???? from to_split_and_scale() ?
# st_scaler # from to_split_and_scale()
# reg_xgb # best model

<a id="5"></a>

# 5. Predicting a price by example features


In [429]:
geo_df_for_check = price_df4.groupby(['city', 'district']).agg({'radius':['min','max']})
geo_df_for_check

radius           
                                                      min        max
city     district                                                   
Kraków   Dzielnica I Stare Miasto                0.067402   2.196423
         Dzielnica II Grzegórzki                 0.782623   3.227532
         Dzielnica III Prądnik Czerwony          2.416610   5.783935
         Dzielnica IV Prądnik Biały              2.370881   6.500390
         Dzielnica IX Łagiewniki-Borek Fałęcki   3.790555   5.847905
         Dzielnica V Krowodrza                   0.931442   3.501961
         Dzielnica VI Bronowice                  2.776384   6.849229
         Dzielnica VII Zwierzyniec               0.880650   9.299495
         Dzielnica VIII Dębniki                  1.031119   9.556217
         Dzielnica X Swoszowice                  6.101539   9.091645
         Dzielnica XI Podgórze Duchackie         3.973828   6.898548
         Dzielnica XII Biezanow-Prokocim         5.338763   9.703491
         Dzielnica XIII Podgórze                 1.895385   9.805905
         Dzielnica XIV Czyżyny                   3.680819   6.437584
         Dzielnica XV Mistrzejowice              4.768365   7.247026
         Dzielnica XVI Bieńczyce                 6.062067   8.379275
         Dzielnica XVII Wzgórza Krzeszławickie   8.194027  11.819953
         Dzielnica XVIII Nowa Huta               6.577547  17.018534
Poznañ   Grunwald                                1.766327   8.946552
         Jeżyce                                  1.395498  12.282787
         Nowe Miasto                             0.840464   9.336733
         Stare Miasto                            0.004959   9.148880
         Wilda                                   1.135756   5.342501
Warszawa Bemowo                                  5.959128   9.614678
         Białołęka                               7.649166  15.255953
         Bielany                                 5.588408  11.244669
         Mokotów                                 2.128140   7.576582
         Ochota                                  0.849038   5.489506
         Praga Południe                          2.991723   7.941227
         Praga Północ                            2.416857   6.364741
         Rembertów                               7.673274  12.219709
         Targówek                                4.858435   9.756911
         Ursus                                   7.657054  11.108546
         Ursynów                                 7.340572  13.508296
         Wawer                                   6.796760  16.319820
         Wesoła                                 11.743397  16.914269
         Wilanów                                 7.206705  12.462417
         Wola                                    0.995535   5.816152
         Włochy                                  4.864296   8.923225
         Śródmieście                             0.147376   3.528454
         Żoliborz                                3.485072   5.684055

In [2]:
def input_to_dict(city, district, floor, rooms, sq, year, radius):
    
    X_check = pd.DataFrame({
        'city': city,
        'district': district,
        'floor': floor, 
        'rooms': rooms, 
        'sq': sq, 
        'year': year,
        'radius': radius
    }, index=[0])
    
    return X_check

In [3]:
# enter your data

input_to_dict('Warszawa', 'Śródmieście', 3, 2, 40, 2000, 2)

,city,district,floor,rooms,sq,year,radius
0,Warszawa,Śródmieście,3,2,40,2000,2


In [450]:
geo_df2 = geo_df_for_check.copy()
geo_df2.columns = ['_'.join(col) for col in geo_df2.columns.values]
geo_df2.reset_index()
geo_df3 = geo_df2.merge(X_check, on=['city','district'])

if len(geo_df3) == 0:
    print('The city or the district is incorrect')
elif not (
    geo_df3.iloc[0]['radius'] >= geo_df3.iloc[0]['radius_min'] 
    and  
    geo_df3.iloc[0]['radius'] <= geo_df3.iloc[0]['radius_max'] 
):
    print("The radius should be between {0} and {1}".format(
        round(geo_df3.iloc[0]['radius_min'],2),
        round(geo_df3.iloc[0]['radius_max'],2)
    ))
else:
    print("Everything is correct")

Everything is correct


In [4]:
def predict_by_input(X_check, cat_features, labels_dict, st_scaler):
    
    for col in cat_features:
        X_check[col] = X_check[col].apply(lambda x: labels_dict[col][x])
#     pd.DataFrame(X_check)

    X_check_scaled = st_scaler.transform(X_check)

    y_check_pred_reg_xgb = reg_xgb.predict(X_check_scaled)
#     y_check_pred_reg_xgb
    
    score = reg_xgb.score(X_test_scaled, y_test)
    price_pred = np.round(y_check_pred_reg_xgb[0])
    print("With a probability of {0}%, the prise will be about {1:,.0f} PLN ".format(
        round(score * 100, 1),
        round(price_pred)
    ))
    
    return price_pred, score

In [462]:
predict_by_input(X_check, cat_features, labels_dict, st_scaler)

With a probability of 87.4%, the prise will be about 631,622 PLN 
